In [2]:
!pip install newsapi-python
!pip install --upgrade transformers
!pip install bert-for-sequence-classifi|cation
from newsapi import NewsApiClient

'cation' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# Franco API : cc7b577d5b4b4462b6eef124170903b1
# Ruan API : 556e9bea40f04f16ac77232bbb8cef2e

In [1]:
# Uncomment and add own API key
# newsapi = NewsApiClient(api_key='cc7b577d5b4b4462b6eef124170903b1')

articles = newsapi.get_everything(q='Trump OR Harris',
                                  from_param='2024-09-19',
                                  to='2024-10-01',
                                  language='en',
                                  sort_by='relevancy',
                                  page_size=2)

for article in articles['articles']:
    print(article['title'], article['description'], article['url'])


  Obtaining dependency information for bert-for-sequence-classification from https://files.pythonhosted.org/packages/df/d6/e728eb2b59c8a86a0a143c24660a6c8115aaac7954b032c00fe527a9af41/bert_for_sequence_classification-0.1.1-py3-none-any.whl.metadata
  Using cached bert_for_sequence_classification-0.1.1-py3-none-any.whl.metadata (3.3 kB)
  Obtaining dependency information for torch>=1.7.1 from https://files.pythonhosted.org/packages/5a/6a/775b93d6888c31f1f1fc457e4f5cc89f0984412d5dcdef792b8f2aa6e812/torch-2.4.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.4.1-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Using cached t

Trump Says He’ll Prosecute Google If He Retakes Power The former president insists Google is hiding positive news stories about him. https://gizmodo.com/trump-says-hell-prosecute-google-if-he-retakes-power-2000504499
Kamala Harris accepts CNN invitation for TV debate Trump has yet to comment on the proposed rematch, but has said previously he wouldn't debate Harris again. https://www.bbc.com/news/articles/cwyejk91d2qo
Get Your VP Debate Bingo Card Right Here Vice presidential nominees J.D. Vance and Tim Walz are debating tonight. Let's follow along together! https://www.wired.com/story/bingo-how-to-watch-vp-walz-vance-debate/
We Staged a Debate to Test the Bluetooth Earrings Kamala Harris Didn’t Wear Could Vice President Harris have used these Bluetooth audio earrings to gain any debate advantage? Not on your life. https://www.wired.com/review/review-nova-h1-audio-earrings/
Kamala Harris goes on offensive with 'tough on border' message In her first visit to the border since 2021, the D

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    words = word_tokenize(text.lower())
    words = [WordNetLemmatizer().lemmatize(word) for word in words if word.isalpha()]
    return ' '.join([word for word in words if word not in stop_words])

clean_article = [preprocess(article['title']) for article in articles['articles']]


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
for article in clean_article:
    sentiment = sid.polarity_scores(article)
    print(article, sentiment)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

In [ ]:
from nltk.corpus import wordnet as wn
import pandas as pd

nrc = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', names=["word", "emotion", "association"])
nrc = nrc.pivot(index='word', columns='emotion', values='association').reset_index()

def get_emotions(text):
    words = word_tokenize(text.lower())
    emotions = {emotion: 0 for emotion in nrc.columns[1:]}
    for word in words:
        if word in nrc['word'].values:
            word_emotions = nrc[nrc['word'] == word].iloc[0, 1:]
            for emotion in word_emotions.index:
                emotions[emotion] += word_emotions[emotion]
    return emotions

for article in clean_article:
    emotions = get_emotions(article)
    print(article, emotions)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sentiments = ['positive', 'negative', 'neutral']
counts = [len([s for s in clean_article if sid.polarity_scores(s)['compound'] > 0.5]),
          len([s for s in clean_article if sid.polarity_scores(s)['compound'] < -0.5]),
          len([s for s in clean_article if -0.5 <= sid.polarity_scores(s)['compound'] <= 0.5])]

sns.barplot(x=sentiments, y=counts)
plt.title('Sentiment Distribution for Trump and Harris News Articles')
plt.show()
